# Transfer learning 

Transfer learning это когда ты берёшь чужую модель и адаптируешь её под свою задачу. В этой тетрадке будет два примера transfer learning. Для эмбедингов и для картинок. 


##  Про tensorhub

Каждый раз, обучая нейронку, мы сначала рандомно инициализируем веса, а после в ходе бэкпропа обучаем модель. Если мы сразу же угадываем хорошие веса, модель сходится быстрее. Иногда можно брать в качестве инициализации веса, полученные другими исследователями и на их основе дообучать модель под свой выход. Это здорово упрощает задачу обучения и экономит многие часы работы. Для такого дележа в $2017$ году была придумана платформа __TensorFlow Hub.__ 

Сегодня с помощью этой платформы разработчики делятся друг с другом уже готовыми предобученными весами. Для работы библиотеки нужна версия tensorflow выше `1.7`. Инструкцию по установке можно найти на [сайте tensorflow.](https://www.tensorflow.org/hub/installation)

В случае проблем с установкой, повысить версию tensorflow до актуальной помогает команда:

```python3 -m pip install --upgrade https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.12.0-py3-none-any.whl```


In [1]:
import tensorflow as tf
import tensorflow_hub as hub

print(tf.__version__)

1.12.0


## 1. ELMO-эмбединги.


[ ](https://simkl.in/fanart/17/17052285f7e1391f7_0.jpg)

Пробуем воспользоваться хабом. Скачаем [модель от IPavlov с эмбедингами для русскоязычных новостей.](http://docs.deeppavlov.ai/en/master/intro/pretrained_vectors.html) Примеры использования хаба с моделями IPavlov можно найти [в документации.](http://docs.deeppavlov.ai/en/latest/apiref/models/embedders.html#deeppavlov.models.embedders.elmo_embedder.ELMoEmbedder)

In [2]:
# подгружаем модель
elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")


INFO:tensorflow:Using /var/folders/r2/6lthpk110g7d1kjwlgx6p7100000gn/T/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz'.
INFO:tensorflow:Downloading http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz: 30.02MB
INFO:tensorflow:Downloading http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz: 60.02MB
INFO:tensorflow:Downloading http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz: 90.02MB
INFO:tensorflow:Downloading http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz: 120.02MB
INFO:tensorflow:Downloading http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz: 150.02MB
INFO:tensorflow:Downloading http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz: 180.02MB
INFO:tensorflow:Downloading http://files.deepp

Запускаем сессию в tensorflow и применяем к чему-нибудь предобученную нейросетку.

In [3]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

embeddings = elmo(["это предложение", "word"], signature="default", as_dict=True)["elmo"]
sess.run(embeddings)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[[ 0.05817385,  0.22493355, -0.1920293 , ..., -0.14448947,
         -0.1242556 ,  1.0148408 ],
        [ 0.5359629 ,  0.28685376,  0.28028587, ..., -0.08028417,
          0.4908908 ,  0.75939935]],

       [[ 0.34336394,  1.0031183 , -0.15972564, ...,  1.2442503 ,
          0.6102935 ,  0.43388352],
        [ 0.05370751,  0.02260921,  0.01074906, ...,  0.08748816,
         -0.0066415 , -0.01344293]]], dtype=float32)

Также hub поддерживает токенизированный формат.

In [4]:
tokens_input = [["мама", "мыла", "раму"], ["рама", "", ""]]
tokens_length = [3, 1]
embeddings = elmo(
    inputs={
            "tokens": tokens_input,
            "sequence_len": tokens_length
            },
    signature="tokens",
    as_dict=True)["elmo"]

sess.run(embeddings)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[[ 0.60400033, -0.16130012,  0.5647884 , ..., -0.00376102,
         -0.0382006 ,  0.26321295],
        [ 0.01834123,  0.17055827,  0.5311495 , ..., -0.56755346,
          0.62669814, -0.05939047],
        [ 0.32425952,  0.17909637,  0.01657113, ...,  0.1866094 ,
          0.7392498 ,  0.08285775]],

       [[ 1.1322286 ,  0.19077665, -0.17811388, ...,  0.42973173,
          0.23391487, -0.01294377],
        [ 0.05370751,  0.02260921,  0.01074906, ...,  0.08748816,
         -0.0066415 , -0.01344293],
        [ 0.05370751,  0.02260921,  0.01074906, ...,  0.08748816,
         -0.0066415 , -0.01344293]]], dtype=float32)

In [9]:
embeddings.shape

TensorShape([Dimension(None), Dimension(None), Dimension(1024)])

Кроме новостей в [документации проекта можно найти](http://docs.deeppavlov.ai/en/master/intro/pretrained_vectors.html) википедию, русскоязычный твиттер и многие другие эмбединги. Там же можно найти довольно много полноценных классификаторов. 

Чуть позже мы попробуем впихнуть предобученный embeddig слой в нашу собственную нейросетку.